In [24]:
from pathlib import Path
from torchvision import transforms
from multiprocessing import *
import numpy
from hashlib import md5
from PIL import Image 
import os
import cv2

path_to_video = "/home/qwest/data_for_ml/DROID/"

data_path = Path(path_to_video)

print(data_path)


/home/qwest/data_for_ml/DROID


In [25]:
image_path_list = list(data_path.glob("*.mp4"))

In [26]:
image_path_list

[PosixPath('/home/qwest/data_for_ml/DROID/00002.mp4'),
 PosixPath('/home/qwest/data_for_ml/DROID/00001.mp4'),
 PosixPath('/home/qwest/data_for_ml/DROID/00003.mp4'),
 PosixPath('/home/qwest/data_for_ml/DROID/00004.mp4'),
 PosixPath('/home/qwest/data_for_ml/DROID/00006.mp4'),
 PosixPath('/home/qwest/data_for_ml/DROID/00005.mp4'),
 PosixPath('/home/qwest/data_for_ml/DROID/00007.mp4')]

In [49]:
transform1 = transforms.Compose([transforms.Resize(64),
                                transforms.CenterCrop(64)])

video_path = image_path_list[8].__str__() 
video = cv2.VideoCapture(video_path)
success = True
count = 1
image_id = 1
while success:
            success , frame = video.read()
            if success == True:
                if count%5 == 0:

                    name = str(md5(frame).hexdigest()[:10])+".jpg"
                    image_id += 1
                    if transform1 is not None:
                        byt = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                        img = transform1(Image.fromarray(byt))
            
                        img.save(f'/home/qwest/data_for_ml/DROID/9/{image_id}.png')
                        #cv2.imwrite(os.path.join('/home/qwest/data_for_ml/test/' , name),frame)
                count += 1
            else:
                break


IndexError: list index out of range

In [32]:
traf(0,2)